In [1]:
import pandas as pd
import gspread
import copy
from oauth2client.service_account import ServiceAccountCredentials

# glibber glabber that I copied and pasted
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('practice-log-444917-d375b6889398.json', scope)
gc = gspread.authorize(credentials)

# spreadsheet_key = '1AgQWFFgXEYRAtokWU8NNDdGKy5R7DYBoPMFukrRtMiA' # This is the test sheet
spreadsheet_key = '13CdEbddbIDDHdCVU9IvpHQzSB4lJroJ36Riufl5uvTk' # This is the real sheet
book = gc.open_by_key(spreadsheet_key)

In [2]:
# Studio in order of grade then alphabetical by last name
studio_names = ['Jaime Cardenas', 'Anna Desfore', 'Matt Schneider', 'Tanner Day', 'Trevor Maliborski', 'Adina Shaikh', 'Dylan Chapman', 'Joshua Dieter',
          'Van Maxwell', 'Cadie Strahota', 'Nate Dreyfus', 'Ravlin Frantz', 'Ellis Goeden', 'Kendal Schreiber', 'Alex Seitler',
          'Cooper Atkinson', 'Aaron Dieter', 'Ryan Gomez', 'Annabelle Nault', 'Marley Van Domelan','Eleanor Peska', 'Aidan Lishner', 'Pete Navratil', 'Nick Ruplinger']

In [3]:
# database of names and nicknames

names = {}
# add first names to database
for full_name in studio_names:
    first_name = full_name.split(' ')[0].lower()
    names[full_name] = [first_name]

names['Cooper Atkinson'].append('coop')
names['Ravlin Frantz'].append('rav')
names['Joshua Dieter'].append('josh')

In [4]:
# grade by name

name_grade = {}

# DMA
for name in ['Jaime Cardenas', 'Anna Desfore', 'Matt Schneider']:
    name_grade[name] = 'DMA'

# Master
for name in ['Tanner Day']:
    name_grade[name] = 'Masters'

# Super Senior
for name in ['Trevor Maliborski', 'Adina Shaikh']:
    name_grade[name] = 'Super Senior'

# Senior
for name in ['Dylan Chapman', 'Joshua Dieter', 'Van Maxwell', 'Cadie Strahota']:
    name_grade[name] = 'Senior'
    
# Junior
for name in ['Nate Dreyfus', 'Ravlin Frantz', 'Ellis Goeden', 'Kendal Schreiber', 'Alex Seitler']:
    name_grade[name] = 'Junior'

# Sophomore
for name in ['Cooper Atkinson', 'Aaron Dieter', 'Ryan Gomez', 'Annabelle Nault', 'Marley Van Domelan', 'Eleanor Peska']:
    name_grade[name] = 'Sophomore'

# Freshman
for name in ['Aidan Lishner', 'Pete Navratil', 'Nick Ruplinger']:
    name_grade[name] = 'Freshman'

In [5]:
timeframes = ['7:00 AM','7:15 AM','7:30 AM','7:45 AM','8:00 AM','8:15 AM','8:30 AM','8:45 AM','9:00 AM','9:15 AM','9:30 AM','9:45 AM', 
'10:00 AM','10:15 AM','10:30 AM','10:45 AM','11:00 AM','11:15 AM','11:30 AM','11:45 AM','12:00 PM','12:15 PM','12:30 PM','12:45 PM','1:00 PM', \
'1:15 PM','1:30 PM','1:45 PM','2:00 PM','2:15 PM','2:30 PM','2:45 PM','3:00 PM','3:15 PM','3:30 PM','3:45 PM','4:00 PM','4:15 PM','4:30 PM','4:45 PM', \
'5:00 PM','5:15 PM','5:30 PM','5:45 PM','6:00 PM','6:15 PM','6:30 PM','6:45 PM','7:00 PM','7:15 PM','7:30 PM','7:45 PM','8:00 PM','8:15 PM','8:30 PM', \
'8:45 PM','9:00 PM','9:15 PM','9:30 PM','9:45 PM','10:00 PM','10:15 PM','10:30 PM','10:45 PM']

In [6]:
# function to determine if a name is in the studio, returns full name or name that was inputted
def name_grabber(cell):
    name = None
    for full_name, names_list in names.items():
        for curr_name in names_list:
            if curr_name == cell:
                name = full_name
        if name != None:
            break
    if name == None:
        return cell
    else:
        return name

In [7]:
# full_df
# takes str room and int week_num and returns the str room_week and a df with filled in gaps
def full_df(room, week_num):
    room_week = (room + " (" + str(week_num) + ")") 
    worksheet = book.worksheet(room_week)
    table = worksheet.get_all_values() # fetch values from worksheet
    
    # Format Table in Pandas
    # Time      Mon   Tue ...
    # 7:00 AM   Josh   '' ...
    # 7:15 AM   Josh   '' ...
    # ...
    df = pd.DataFrame(table[3:67])
    df.columns = ['Time', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    
    for day in df:
        series = df[day]
        for idx in range(len(series)):
            cell = series.iloc[idx]
            if cell != '':   # "if cell is not empty..."
                diff = 1
                for count in range(1,9):
                    if (idx + count) > (len(series) - 1):   # max bounds case
                        break
                    next_cell = series.iloc[idx + count]
                    if (next_cell != cell) and (next_cell != ''):
                        break
                    if next_cell == cell:
                        diff = count
                        break
                for i in range(diff):
                    df.loc[i + idx, day] = cell
                    
    df = df.set_index('Time').rename_axis(None)
    return room_week, df

In [8]:
# rooms
rooms = ["Studio", "1401", "1407", "1409", "1413", "1414", "1416", "1417", "1418"]

In [9]:
# compile dataframes, don't run too much or will overload page
compiled_data = {}
for room in rooms:
    for week_num in range(1,3):
        room_week, df = full_df(room, week_num)
        compiled_data[room_week] = df
# compiled_data

In [11]:
# summarizing the data

entries_by_person = {}
other_entries = []
entries_by_person_by_time = {}
entries_by_time = {}

entries_by_room = {}
entries_by_room_wo_lessons = {}

for room, df in compiled_data.items():
    # initialize total entries by room
    if room not in entries_by_room:
        entries_by_room[room] = 0
        entries_by_room_wo_lessons[room] = 0
    
    # total entries by person, other entries data
    for day in df:
        series = df[day]
        for time, cell in series.items():
            lcell = cell.lower().strip()
            if lcell == '':   # if cell is empty, skip to next cell
                continue
            full_name = name_grabber(lcell) # grabs full name

            # save entries who are not in studio
            if full_name == lcell:
                if full_name not in other_entries:
                    other_entries.append(full_name)
                    
            # add to total entries by person dict
            # and initialize entries_by_person_by_time
            if full_name not in entries_by_person:
                entries_by_person[full_name] = 0
                entries_by_person_by_time[full_name] = {}
            entries_by_person[full_name] += 1

            if time not in entries_by_person_by_time[full_name]:
                entries_by_person_by_time[full_name][time] = 0
            entries_by_person_by_time[full_name][time] += 1

            entries_by_room[room] += 1 # add 1 to room
            if lcell != 'lessons':
                entries_by_room_wo_lessons[room] += 1 # add 1 to room (wo lessons)

    # counting how many entries are in each 15min timeframe
    for time in timeframes:
        time_series = df.loc[time]
        if time not in entries_by_time:
            entries_by_time[time] = 0
        for cell in time_series:
            if cell != '':
                entries_by_time[time] += 1

entries_by_room, entries_by_room_wo_lessons

({'Studio (1)': 38,
  'Studio (2)': 192,
  '1401 (1)': 19,
  '1401 (2)': 188,
  '1407 (1)': 3,
  '1407 (2)': 129,
  '1409 (1)': 37,
  '1409 (2)': 241,
  '1413 (1)': 10,
  '1413 (2)': 121,
  '1414 (1)': 12,
  '1414 (2)': 171,
  '1416 (1)': 0,
  '1416 (2)': 8,
  '1417 (1)': 10,
  '1417 (2)': 139,
  '1418 (1)': 3,
  '1418 (2)': 35},
 {'Studio (1)': 38,
  'Studio (2)': 139,
  '1401 (1)': 19,
  '1401 (2)': 188,
  '1407 (1)': 3,
  '1407 (2)': 129,
  '1409 (1)': 37,
  '1409 (2)': 205,
  '1413 (1)': 10,
  '1413 (2)': 121,
  '1414 (1)': 12,
  '1414 (2)': 171,
  '1416 (1)': 0,
  '1416 (2)': 8,
  '1417 (1)': 10,
  '1417 (2)': 139,
  '1418 (1)': 3,
  '1418 (2)': 35})

In [ ]:
# Grab that data (OLD VERSION)

# Get different sheets
rooms = ["Studio", "1401", "1407", "1409", "1413", "1414", "1416", "1417", "1418"]

entries_by_person = {}
entries_by_timeframe = {}
entries_by_person_timeframe = {}
entries_by_room = {}
entries_by_room_wo_lessons = {}
other_entries = []
for room in rooms:
    for week in range(1,3):
        room_week = (room + " (" + str(week) + ")")
        worksheet = book.worksheet(room_week)
        table = worksheet.get_all_values()
        
        # Format Table in Pandas
        # Time      Mon   Tue ...
        # 7:00 AM   Josh   '' ...
        # 7:15 AM   Josh   '' ...
        # ...
        df = pd.DataFrame(table[3:67])
        df.columns = ['Time', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        
        # Modify dataframe so it fills in gaps in current dataframe
        for day in df:
            for idx in range(len(df[day])):
                cell = df[day].iloc[idx]
                if cell != '':
                    diff = 1
                    for idx2 in range(1,9):
                        if (idx+idx2) > (len(df[day])-1):   # max bounds case
                            break
                        next_cell = df[day].iloc[idx+idx2]
                        if (next_cell != cell) and (next_cell != ''):
                            break
                        if next_cell == cell:
                            diff = idx2
                            break
                    for i in range(diff):
                        df.loc[i+idx,day] = cell
                        
        # Indexes aren't needed anymore so setting 'Time' as index
        df = df.set_index('Time').rename_axis(None)
        
        # total entries by person
        # and entries by person by timeframe
        for day in df:
            for timeframe_cell in df[day].items():
                timeframe = timeframe_cell[0]
                cell = timeframe_cell[1]
                lcell = cell.lower().strip()
                if lcell == '':   # if cell is empty, skip to next cell
                    continue
                full_name = name_grabber(lcell) # grabs full name

                # other entries
                if full_name == lcell:
                    if full_name not in other_entries:
                        other_entries.append(full_name)

                # entries by person
                if full_name not in entries_by_person:
                    entries_by_person[full_name] = 0
                    entries_by_person_timeframe[full_name] = {}
                entries_by_person[full_name] += 1

                # entries by person by timeframe
                if timeframe not in entries_by_person_timeframe[full_name]:
                    entries_by_person_timeframe[full_name][timeframe] = 0
                entries_by_person_timeframe[full_name][timeframe] += 1
                
                
        # total entries by room
        if room not in entries_by_room:
            entries_by_room[room] = 0
        for day in df:
            for cell in df[day]:
                if cell != '':
                    entries_by_room[room] += 1

        # total entries by room without lessons
        if room not in entries_by_room_wo_lessons:
            entries_by_room_wo_lessons[room] = 0
        for day in df:
            for cell in df[day]:
                if cell != '' and cell != 'LESSONS':
                    entries_by_room_wo_lessons[room] += 1

        # counting how many entries are in each 15min timeframe
        for timeframe in timeframes:
            timeframe_series = df.loc[timeframe]
            for cell in timeframe_series:
                if cell != '':
                    if timeframe not in entries_by_timeframe:
                        entries_by_timeframe[timeframe] = 0
                    entries_by_timeframe[timeframe] += 1
                    
        
# sorted_entries_by_person = dict(sorted(entries_by_person.items(), key=lambda ele: ele[1], reverse = True))
# sorted_entries_by_room = dict(sorted(entries_by_room.items(), key=lambda ele: ele[1], reverse = True))
# sorted_entries_by_room_wo_lessons = dict(sorted(entries_by_room_wo_lessons.items(), key=lambda ele: ele[1], reverse = True))

In [ ]:
# sorted_entries_by_timeframe = dict(sorted(entries_by_timeframe.items(), key=lambda ele: ele[1], reverse = True))
# # sorted_entries_by_timeframe

In [12]:
# make separate lists for names in studio and other

entries_by_studio = copy.deepcopy(entries_by_person)
entries_by_other = {}
for person_entries in entries_by_person.items():
    if person_entries[0] in other_entries:
        entries_by_other[person_entries[0]] = entries_by_studio.pop(person_entries[0])

entries_by_other

{'ganner': 1,
 'lessons': 89,
 'rep': 6,
 'lesson': 1,
 'tanner lesson': 4,
 'ellis lesson': 22,
 'aed': 11,
 'tom': 12,
 'marley (lesson)': 4,
 'coop/aaron': 5,
 'eleanor lesson': 8,
 'alex lesson': 4,
 'rav lesson': 1,
 'cooper lesson': 9,
 'maey': 1,
 'nate lesson': 2,
 'aaron lesson': 8,
 'kendal lesson': 2,
 'vibe out': 26,
 'vibe back': 1,
 'cadie lesson': 6,
 'marimba out': 44,
 'cadie lesson (marimba out)': 4,
 '(pete) mba out': 6,
 'vibe out (josh)': 2,
 'msrley': 2}

In [ ]:
# get the stats worksheet
# stats = book.worksheet('Stats')

In [ ]:
# # individual stats
# time_by_studio = entries_to_time(entries_by_studio)
# time_by_other = entries_to_time(entries_by_other)

# # add to leaderboard with studio only values
# indiv_idx = 5
# for person_time in time_by_studio.items():
#     person = person_time[0]
#     time = person_time[1]
#     stats.update_acell('A'+str(indiv_idx),person)
#     stats.update_acell('B'+str(indiv_idx),time)
#     indiv_idx += 1

# # add to other with other entries
# indiv_idx = 5
# for person_time in time_by_other.items():
#     person = person_time[0]
#     time = person_time[1]
#     stats.update_acell('C'+str(indiv_idx),person)
#     stats.update_acell('D'+str(indiv_idx),time)
#     indiv_idx += 1

In [ ]:
# # add room stats

# # first, convert 15 min entries to hr min
# time_by_room = entries_to_time(entries_by_room)
# time_by_room_wo_lessons = entries_to_time(entries_by_room_wo_lessons)

# # with lessons
# room_idx = 5
# for room_time in time_by_room.items():
#     room = room_time[0]
#     time = room_time[1]
#     stats.update_acell('E'+str(room_idx),room)
#     stats.update_acell('F'+str(room_idx),time)
#     room_idx += 1

# # without lessons
# room_idx = 5
# for room_time in time_by_room_wo_lessons.items():
#     room = room_time[0]
#     time = room_time[1]
#     stats.update_acell('G'+str(room_idx),room)
#     stats.update_acell('H'+str(room_idx),time)
#     room_idx += 1

In [13]:
# entries to time conversion function
def entries_to_time(row):
    hours = row['Entries']*15/60
    hours_mins = str(hours).split('.')
    if int(hours_mins[0]) == 0:
        return str(int(float('.'+hours_mins[1])*60)) + 'mins'
    else: 
        return str(hours_mins[0]) + 'hrs ' + str(int(float('.'+hours_mins[1])*60)) + 'mins'

In [14]:
# entries to time per day (assume 7 days a week)

def entries_to_time_p_day(row):
    hours = row['Entries']*15/60/14  # divide by the 7 days in the week
    hours_mins = str(hours).split('.')
    if int(hours_mins[0]) == 0:
        return str(int(float('.'+hours_mins[1])*60)) + 'mins'
    else: 
        return str(hours_mins[0]) + 'hrs ' + str(int(float('.'+hours_mins[1])*60)) + 'mins'

In [15]:
# fav timeframe function
def fav_timeframe(row):
    for name, dict in entries_by_person_by_time.items():
        if name == row['Name']:
            return max(dict, key=dict.get)
    return 'N/A'

In [16]:
# dataframe with all the stats sorted in alphabetical order by name

# take list of names and entry amounts (excluding other, also unsorted) and convert to dataframe, add other onto the end so its more organized

# combine studio and other entries
entries_by_combined = entries_by_studio|entries_by_other
# s_entries_by_combined = dict(sorted(entries_by_combined.items(), key=lambda ele: ele[0], reverse = False))

name_entries_df = pd.DataFrame.from_dict(entries_by_combined, orient="index").reset_index()
name_entries_df.columns = ['Name','Entries']

# add grade level
name_grade_df = pd.DataFrame.from_dict(name_grade, orient='index').reset_index()
name_grade_df.columns = ['Name','Grade']

# merge entries and grade level
perc_stats = pd.merge(name_grade_df, name_entries_df, on='Name',how='outer')

# fill NaN
perc_stats.fillna({'Grade': 'Unknown', 'Entries': 0}, inplace=True)

# Time Practiced
perc_stats['Time Practiced'] = perc_stats.apply(entries_to_time, axis=1)

# Time/Day
perc_stats['Time per Day (14)'] = perc_stats.apply(entries_to_time_p_day, axis=1)

# Favorite Practice Timeframe
perc_stats['Favorite Practice Timeframe'] = perc_stats.apply(fav_timeframe, axis=1)

sorted_perc_stats = perc_stats.sort_values('Entries', ascending = False)

# sorted_perc_stats[sorted_perc_stats['Grade'] == 'Junior']

sorted_perc_stats.groupby('Grade').sum('Entries')

entries_by_grade = sorted_perc_stats.groupby('Grade').agg(
    Count=('Entries', 'count'),
    Entries=('Entries', 'sum')
)

entries_by_grade['avg'] = entries_by_grade['Entries']/entries_by_grade['Count']
entries_by_grade

# perc_stats

,Count,Entries,avg
Grade,,,
DMA,3,42.0,14.000000
Freshman,3,146.0,48.666667
Junior,5,353.0,70.600000
Masters,1,18.0,18.000000
Senior,4,116.0,29.000000
Sophomore,6,313.0,52.166667
Super Senior,2,87.0,43.500000
Unknown,26,281.0,10.807692


In [17]:
sorted_perc_stats[sorted_perc_stats["Grade"] != "Unknown"]

,Name,Grade,Entries,Time Practiced,Time per Day (14),Favorite Practice Timeframe
11,Ellis Goeden,Junior,148.0,37hrs 0mins,2hrs 38mins,2:30 PM
19,Pete Navratil,Freshman,72.0,18hrs 0mins,1hrs 17mins,1:15 PM
20,Ravlin Frantz,Junior,70.0,17hrs 30mins,1hrs 15mins,3:45 PM
6,Annabelle Nault,Sophomore,67.0,16hrs 45mins,1hrs 11mins,5:00 PM
15,Marley Van Domelan,Sophomore,66.0,16hrs 30mins,1hrs 10mins,11:00 AM
1,Aaron Dieter,Sophomore,62.0,15hrs 30mins,1hrs 6mins,3:00 PM
4,Alex Seitler,Junior,50.0,12hrs 30mins,53mins,8:30 AM
14,Kendal Schreiber,Junior,48.0,12hrs 0mins,51mins,4:30 PM
10,Eleanor Peska,Sophomore,47.0,11hrs 45mins,50mins,11:00 AM
2,Adina Shaikh,Super Senior,46.0,11hrs 30mins,49mins,4:30 PM
